In [ ]:
%matplotlib notebook

from libsignetsim.model.SbmlDocument import SbmlDocument
from libsignetsim.model.sbml.KineticLaw import KineticLaw
from libsignetsim.simulation.TimeseriesSimulation import TimeseriesSimulation
from libsignetsim.data.Experiment import Experiment
from libsignetsim.optimization.ModelVsTimeseriesOptimization import ModelVsTimeseriesOptimization

from os.path import join
import matplotlib.pyplot as plt

In [ ]:
# Loading the model
sbml_filename = join("sbml_files", "ras_mapk.xml")
sbml_doc = SbmlDocument()
sbml_doc.readSbmlFromFile(sbml_filename)
model = sbml_doc.getModelInstance()

# Settings the FGF2 stimulus
model.listOfSpecies.getBySbmlId("fgf2").setValue(333)

In [ ]:
# Simulating the model
sim = TimeseriesSimulation([model], time_min=0, time_ech=6, time_max=3600)
sim.run()
ts, ys = sim.getRawData()[0]

In [ ]:
# Building the fake experimental data from the simulated trajectories
experiment = Experiment()
condition = experiment.createCondition()

for i, t in enumerate(ts):
    condition.addObservation(t, "Ras-GTP", ys["ras_gtp"][i])
    condition.addObservation(t, "Mapk-PP", ys["mapk_pp"][i])

In [ ]:
# Removing the original SOS activation by FGF2

reaction_fgf = model.listOfReactions.values()[6]
print reaction_fgf
model.listOfReactions.remove(reaction_fgf)

In [ ]:
# Adding a new activation mechanism. 

# First reaction activates SOS as an enzymatic reaction 
kcat = model.listOfParameters.new("kcat")
km = model.listOfParameters.new("km")
kdeg = model.listOfParameters.new("kdeg")

reaction_activation_sos = model.listOfReactions.new()
reaction_activation_sos.listOfReactants.add(model.listOfSpecies.getBySbmlId("sos_i"))
reaction_activation_sos.listOfModifiers.add(model.listOfSpecies.getBySbmlId("fgf2"))
reaction_activation_sos.listOfProducts.add(model.listOfSpecies.getBySbmlId("sos"))
reaction_activation_sos.setKineticLaw(KineticLaw.MICHAELIS, reversible=False, parameters=[kcat, km])
print reaction_activation_sos

# Second reaction desactivate fgf2 with time
reaction_degradation_fgf2 = model.listOfReactions.new()
reaction_degradation_fgf2.listOfReactants.add(model.listOfSpecies.getBySbmlId("fgf2"))
reaction_degradation_fgf2.setKineticLaw(KineticLaw.MASS_ACTION, reversible=False, parameters=[kdeg])
print reaction_degradation_fgf2

In [ ]:
# Simulating the new mechanism

sim_mod = TimeseriesSimulation([model], time_min=0, time_ech=6, time_max=3600)
sim_mod.run()
ts_mod, ys_mod = sim_mod.getRawData()[0]

In [ ]:
# Plotting the trajectories for comparison

plt.figure()
plot1 = plt.subplot(2,1,1)
plot1.plot(ts, ys["ras_gtp"], ts, ys["mapk_pp"])
plot2 = plt.subplot(2,1,2)
plot2.plot(ts_mod, ys_mod["ras_gtp"], ts_mod, ys_mod["mapk_pp"])

In [ ]:
# Selecting the parameters from the new mechanism 
selected_parameters = []
selected_parameters.append((kcat, 1, 1e-8, 1e+8))
selected_parameters.append((km, 1, 1e-8, 1e+8))
selected_parameters.append((kdeg, 1, 1e-8, 1e+8))

# Lauching the fitting
fit = ModelVsTimeseriesOptimization(
    workingModel=model,
    list_of_experiments=[experiment],
    parameters_to_fit=selected_parameters,
    p_lambda=0.001
)

# Running on 2 processors
score = fit.runOptimization(2)
parameters = fit.readOptimizationOutput()

In [ ]:
# Setting the parameters values from the fit results

kcat.setValue(parameters[kcat])
km.setValue(parameters[km])
kdeg.setValue(parameters[kdeg])

sim_fitted = TimeseriesSimulation([model], time_min=0, time_ech=6, time_max=3600)
sim_fitted.run()
ts_fitted, ys_fitted = sim_fitted.getRawData()[0]

In [ ]:
# Plotting the original and fitted model

plt.figure()
plot1 = plt.subplot(2,1,1)
plot1.plot(ts, ys["ras_gtp"], ts, ys["mapk_pp"])
plot2 = plt.subplot(2,1,2)
plot2.plot(ts_fitted, ys_fitted["ras_gtp"], ts_fitted, ys_fitted["mapk_pp"])